In [1]:
import sys
sys.path.insert(1, '/media/cinthia/Dados/Mestrado/mv-text-summarizer')

import glob, os
import pandas as pd
import json
import spacy
import nltk
import numpy as np
import json
import smogn
import seaborn as sns
import pickle

from bs4 import BeautifulSoup
from pysbd.utils import PySBDFactory
import math

from sumeval.metrics.rouge import RougeCalculator
rouge = RougeCalculator(stopwords=True, lang="en")
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from timeit import default_timer as timer 
from imblearn.over_sampling import SMOTE
from sklearn.metrics import matthews_corrcoef
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from keras.models import model_from_json

#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#nltk.download('punkt')
#nltk.download('stopwords')

#!python -m spacy download en_core_web_sm
nlp_sm = spacy.load('en_core_web_sm')

#!python -m spacy download en_core_web_md
nlp_md = spacy.load('en_core_web_md')

import warnings
warnings.filterwarnings("ignore")

path_base = "../../sumdata/dataset_articles"


In [2]:
%load_ext autoreload
%autoreload 2

from src import preprocess
from src import extract_features
from src import tokenizer
from src import create_features_df
from src import transform_data
from src import loader
from src import utils
from src import ensemble_tree_models
from src import tunning_hyperparametrs as th
from src import mlp_regressor
from src import mlp_classifier
from src import summarization
from src import normalization
from src import ensemble_tree_models as classifiers
from src import utils_classification as utils_clf
from src import evaluate_classifiers as ev
from src import prepare_data

# Experimentos e Resultados

## Extração das Features

In [ ]:
def main(name_section, initial_batch, embed_dim, batch_len, path_basem verbose):
    
    features_columns = ['sentences', 'text_rank', 'lex_rank', 'count_one_gram', 'count_two_gram',
       'count_three_gram', 'count_article_keywords', 'tf-isf',
       'position_score', 'paragraph_score', 'number_citations', 'length_score',
       'pos_score', 'ner_score', 'dist_centroid', 'articles']
    
    scores_columns = ['rouge_1', 'rouge_2', 'rouge_l', 'label', 'articles']
    
    embeddings_columns = [i for i in range(embed_dim)]
    embeddings_columns.append("article")
    

    if os.path.isfile('batches_{}.json'.format(batch_len)):

        print("Load Batch Files")
        with open('batches_{}.json'.format(batch_len) as f:
            batches = json.load(f)

        batch_files = [value for key, value in batches.items()]
    else: 

        print("Creating Batch Files")
        batch_files = create_batches(path_base, tam=batch_len)
        utils.save_batches(batch_files)
        

    batche_files = batch_files[args.initial_batch:]
    extract_features_batches(
        batch_files, path_base, name_section=name_section, initial_batch=initial_batch, verbose=verbose)

In [ ]:
initial_batch=36
batche_files = batch_files[initial_batch:]

path_base = "../../sumdata/dataset_articles"
extract_features_batches(batch_files, path_base, name_section='conclusion', initial_batch=initial_batch, verbose=True)

Name section: conclusion
Iniciando a extração de features...
Batch: 36 

Total de arquivos: (159, 4) 

Time: 1003.707691265
Quantidade de arquivos processados: 159
Saving Results
Batch: 37 

Total de arquivos: (159, 4) 

Time: 1044.7837589849996
Quantidade de arquivos processados: 159
Saving Results
Batch: 38 

Total de arquivos: (159, 4) 

Time: 1038.4883319669998
Quantidade de arquivos processados: 159
Saving Results
Batch: 39 

Total de arquivos: (159, 4) 

Time: 984.2681289150005
Quantidade de arquivos processados: 159
Saving Results
Batch: 40 

Total de arquivos: (159, 4) 

Time: 987.2068271289991
Quantidade de arquivos processados: 159
Saving Results
Batch: 41 

Total de arquivos: (159, 4) 

Time: 1069.3134625249986
Quantidade de arquivos processados: 159
Saving Results
Batch: 42 

Total de arquivos: (159, 4) 

Time: 1026.3326052150005
Quantidade de arquivos processados: 159
Saving Results
Batch: 43 

Total de arquivos: (159, 4) 

Time: 1074.9373005910002
Quantidade de arquivos p

# Load Features

In [7]:
#import random

#valid_len = int(len(comuns)*0.2)
#summ_items = random.sample(comuns, valid_len)

#df = pd.DataFrame({'summ': summ_items})
#df.to_csv("indices_summ.csv", index=False)

In [ ]:
def create_data_regression(dataset):

    columns_name = ['text_rank', 'lex_rank', 'count_one_gram',
        'count_article_keywords',
       'tf-isf', 'position_score', 'paragraph_score',
       'length_score', 'pos_score', 'ner_score', 'dist_centroid']

    features, scores = loader.read_features(path="../result/{}/features_*.csv".format(dataset))

    summ_items = list(pd.read_csv("indices_summ.csv")['summ'])

    data = utils.join_dataset(features, scores)
    train, test = utils.split_dataset (data, summ_items)

    X_train, scaler = normalization.scale_fit_transform(X_train, section='scaler_{}_regress'.format(dataset))
    y_train = train['rouge_1']*100
    X_train, y_train = utils_clf.shuffle_dataset(X_train, y_train)
    
    X_test = scaler.transform(test[columns_name])
    y_test = test['rouge_1']*100
    
    return X_train, X_test, y_train, y_test

In [3]:
def main(path='.', format_dataset=True, verbose=True):
    
    columns_name = ['text_rank', 'lex_rank', 'count_one_gram',
        'count_article_keywords', 'tf-isf', 'position_score', 'paragraph_score',
       'length_score', 'pos_score', 'ner_score', 'dist_centroid']

    sections=['introduction', 'materials', 'conclusion', 'concat']
    #sections=[ 'introduction']

    if format_dataset:
        if verbose:
            print("Preparando dataset para os classificadores")
        dataset = prepare_data.main_create_dataset(columns_name, sections)
        #utils.save_json(dataset, name='dataset', path=path)
    else:
        if verbose:
            print("Carregando dataset")
        dataset = utils.load_json(name='dataset', path=path)
    
    if verbose:
        print("Treinamento dos modelos")
        
    return dataset
    
    '''models = ensemble_tree_models.create_models(dataset, sections, name_models=['knn', 'gb', 'rf', 'ab'])
    predictions, results = ev.create_reports(models, dataset, columns_name, verbose=False)
    
    #utils.save_json(predictions, name='prediction', path=path)
    #utils.save_results(results, path=path)
    
    parameters = {'introduction': [0.2, 100, 64],
             'materials': [0.2, 100, 64],
             'conclusion':[0.2, 100, 64],
             'concat': [0.2, 100, 64]}

    models_nn = mlp_classifiers.main_train_nn(dataset, sections, parameters, train=True, verbose=False)
    predictions, results =  mlp_classifiers.eval_nn(dataset, sections)'''

In [4]:
dataset = main(verbose=True)

Preparando dataset para os classificadores
Treinamento dos modelos


In [5]:
columns_name = ['text_rank', 'lex_rank', 'count_one_gram',
        'count_article_keywords', 'tf-isf', 'position_score', 'paragraph_score',
       'length_score', 'pos_score', 'ner_score', 'dist_centroid']

sections=['introduction', 'materials', 'conclusion', 'concat']

In [ ]:
models = ensemble_tree_models.create_models(dataset, sections, name_models=['knn', 'gb', 'rf', 'ab'])
predictions, results = ev.create_reports(models, dataset, columns_name, verbose=False)

In [ ]:
parameters = {'introduction': [0.2, 100, 64],
             'materials': [0.2, 100, 64],
             'conclusion':[0.2, 100, 64],
             'concat': [0.2, 100, 64]}

models_nn = mlp_classifiers.main_train_nn(dataset, sections, parameters, train=True, verbose=False)
predictions, results =  mlp_classifiers.eval_nn(dataset, sections)

In [198]:
dataset = utils.load_json(name='dataset', path='.')

In [ ]:
if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument('--path', "-p",required=True)
    parser.add_argument('--format_dataset', "-f",required=True)
     
    args = parser.parse_args()

    columns_name = ['text_rank', 'lex_rank', 'count_one_gram',
        'count_article_keywords',
       'tf-isf', 'position_score', 'paragraph_score',
       'length_score', 'pos_score', 'ner_score', 'dist_centroid']

    sections=['introduction', 'materials', 'conclusion', 'concat']

   

In [ ]:
from sklearn.cluster import DBSCAN
>>> import numpy as np
>>> X = np.array([[1, 2], [2, 2], [2, 3],
...               [8, 7], [8, 8], [25, 80]])
>>> clustering = DBSCAN(eps=3, min_samples=2).fit(X)
>>> clustering.labels_

In [ ]:
from sklearn.cluster import DBSCAN

clustering = DBSCAN(eps=3, min_samples=2).fit(dataset)
lustering.labels_

# Regression Models

In [14]:
from sklearn import svm
from sklearn import neighbors
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor

In [15]:
def pipeline_regression(dataset_name, X, y):
    
    lr = LinearRegression().fit(X, y)
    knn = KNeighborsRegressor(n_neighbors=5).fit(X, y)
    #svm = SVC().fit(X, y)
    gbr = classifiers.fit_model(name_model='gb_r', dataset=dataset_name, X_train=X, y_train=y,
                                n_estimators=50, min_samples_leaf=100, min_samples_split=200, max_depth=20)
    rfr = classifiers.fit_model(name_model='rf_r', dataset=dataset_name, X_train=X, y_train=y,
                                n_estimators=50, min_samples_leaf=100, min_samples_split=200, max_depth=20)
    
    return lr, knn, gbr, rfr

In [21]:
lr_intro, knnr_intro, gbr_intro, rfr_intro = pipeline_regression('intro', X_train_intro, y_train_intro)
lr_mat, knnr_mat, gbr_mat, rfr_mat = pipeline_regression('mat', X_train_mat, y_train_mat)
lr_conc, knnr_conc, gbr_conc, rfr_conc = pipeline_regression('conc', X_train_conc, y_train_conc)
lr_concat, knnr_concat, gbr_concat, rfr_concat = pipeline_regression('concat', X_concat_train, y_concat_train)

## MLP

In [342]:
mlp.train_neural_model(
    dataset='intro', view='features', X_train=X_train_intro, y_train= y_train_intro, input_shape=11,
    validation_split=0.2, learning_rate=0.01,
         epochs=300, batch_size=64, verbose=False)

Saved model to disk


In [345]:
mlp.train_neural_model(
    dataset='mat', view='features', X_train=X_train_mat, y_train= y_train_mat, input_shape=11,
    validation_split=0.2, learning_rate=0.01,
         epochs=300, batch_size=64, verbose=False)

Saved model to disk


In [346]:
mlp.train_neural_model(
    dataset='conc', view='features', X_train=X_train_conc, y_train= y_train_conc, input_shape=11,
    validation_split=0.2, learning_rate=0.01,
         epochs=300, batch_size=64, verbose=False)


Saved model to disk


In [347]:
mlp.train_neural_model(
    dataset='concat', view='features', X_train=X_concat_train, y_train= y_concat_train, input_shape=11,
    validation_split=0.2, learning_rate=0.01,
         epochs=300, batch_size=64, verbose=False)

Saved model to disk


## Summarization

In [22]:
models = {'lr':[lr_intro, lr_mat, lr_conc, lr_concat],
    'knn': [knnr_intro, knnr_mat, knnr_conc, knnr_concat],
    'gb': [gbr_intro, gbr_mat, gbr_conc, gbr_concat],
    'rf': [rfr_intro, rfr_mat, rfr_conc, rfr_concat]}

In [23]:
dataset = {'intro': [X_summ_intro, y_summ_intro], 
     'mat': [X_summ_mat, y_summ_mat], 
     'conc': [X_summ_conc, y_summ_conc]}

In [24]:
dataset['intro'].append(pickle.load(open('scale_{}.pkl'.format('intro'), 'rb')))
dataset['mat'].append(pickle.load(open('scale_{}.pkl'.format('mat'), 'rb')))
dataset['conc'].append(pickle.load(open('scale_{}.pkl'.format('conc'), 'rb')))

In [25]:
def create_summaries(r, dataset, columns_name, summ_items, path_base):

    results = {}

    for i in r.keys():

        models = r[i]

        summaries, intro, mat, conc = summarization.pipeline_summarization(
            i, models, dataset, columns_name, summ_items, path_base, sections=['intro', 'mat', 'conc', 'comb'])

        results[i] = [summaries, intro, mat, conc]

    return results
    

In [26]:
summaries = create_summaries(models, dataset, columns_name, summ_items, path_base)

## Results

In [324]:
summaries['lr'][0].describe()

,intro_r1,intro_r2,intro_rl,mat_r1,mat_r2,mat_rl,conc_r1,conc_r2,conc_rl,concat_r1,concat_r2,concat_rl
count,442.000000,442.000000,442.000000,442.000000,442.0,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000
mean,0.194058,0.070973,0.135773,0.109460,0.0,0.085101,0.145153,0.034170,0.098801,0.248285,0.080143,0.147645
std,0.086495,0.064422,0.066263,0.087211,0.0,0.070157,0.076419,0.040349,0.051966,0.089790,0.073903,0.072677
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.134669,0.030189,0.095238,0.045767,0.0,0.041505,0.090295,0.008475,0.064132,0.192424,0.040468,0.106515
50%,0.189039,0.057708,0.124647,0.097391,0.0,0.072690,0.135375,0.023393,0.088799,0.241822,0.064020,0.136037
75%,0.245761,0.094376,0.170294,0.148838,0.0,0.109299,0.197613,0.047621,0.126374,0.295584,0.095501,0.169324
max,0.583851,0.490566,0.559006,0.448598,0.0,0.403162,0.444444,0.404494,0.444444,0.697872,0.609442,0.655319


In [325]:
summaries['knn'][0].describe()

,intro_r1,intro_r2,intro_rl,mat_r1,mat_r2,mat_rl,conc_r1,conc_r2,conc_rl,concat_r1,concat_r2,concat_rl
count,442.000000,442.000000,442.000000,442.000000,442.0,442.000000,442.000000,442.0,442.000000,442.000000,442.000000,442.000000
mean,0.191479,0.060221,0.127736,0.070825,0.0,0.057321,0.145824,0.0,0.103902,0.251737,0.075088,0.149548
std,0.077759,0.058224,0.057450,0.070325,0.0,0.057498,0.082883,0.0,0.056607,0.087705,0.074684,0.073820
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,0.136519,0.023952,0.091013,0.017897,0.0,0.017739,0.086589,0.0,0.064206,0.197789,0.036890,0.113636
50%,0.183383,0.046225,0.120109,0.057143,0.0,0.046620,0.143646,0.0,0.103045,0.244966,0.059652,0.137913
75%,0.235294,0.078377,0.153636,0.095676,0.0,0.076801,0.198792,0.0,0.134720,0.300654,0.094716,0.173067
max,0.494737,0.489362,0.494737,0.447489,0.0,0.383562,0.450704,0.0,0.444444,0.697872,0.609442,0.655319


In [326]:
summaries['gb'][0].describe()

,intro_r1,intro_r2,intro_rl,mat_r1,mat_r2,mat_rl,conc_r1,conc_r2,conc_rl,concat_r1,concat_r2,concat_rl
count,442.000000,442.000000,442.000000,442.000000,442.0,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000
mean,0.188994,0.060542,0.127675,0.081096,0.0,0.063755,0.159980,0.043281,0.111020,0.251597,0.076973,0.151307
std,0.076994,0.056720,0.059202,0.080131,0.0,0.064210,0.092191,0.053815,0.065875,0.091665,0.075873,0.076669
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.137657,0.024730,0.086957,0.023988,0.0,0.021478,0.093023,0.010230,0.069782,0.193722,0.036675,0.111111
50%,0.179443,0.048193,0.118785,0.061856,0.0,0.050569,0.154643,0.027397,0.104870,0.242427,0.059353,0.137791
75%,0.233296,0.079298,0.151803,0.111801,0.0,0.084619,0.221995,0.063456,0.145068,0.302510,0.094003,0.170758
max,0.494737,0.489362,0.494737,0.453333,0.0,0.382514,0.657005,0.614634,0.647343,0.832061,0.753846,0.793893


In [323]:
summaries['rf'][0].describe()

,intro_r1,intro_r2,intro_rl,mat_r1,mat_r2,mat_rl,conc_r1,conc_r2,conc_rl,concat_r1,concat_r2,concat_rl
count,442.000000,442.000000,442.000000,442.000000,442.0,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000
mean,0.186354,0.061687,0.125765,0.077678,0.0,0.060802,0.150430,0.039989,0.104882,0.246243,0.076346,0.148001
std,0.078642,0.057585,0.058774,0.076780,0.0,0.060802,0.093198,0.053635,0.065212,0.096258,0.079698,0.080045
min,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.129368,0.024578,0.085323,0.017560,0.0,0.016129,0.085027,0.000000,0.063771,0.182811,0.035438,0.107323
50%,0.177380,0.048780,0.117047,0.058252,0.0,0.050033,0.142857,0.025040,0.098160,0.237456,0.059702,0.136293
75%,0.235790,0.081456,0.153846,0.112322,0.0,0.085757,0.207385,0.057554,0.134510,0.291624,0.094062,0.167247
max,0.494737,0.489362,0.494737,0.388889,0.0,0.382514,0.657005,0.614634,0.647343,0.832061,0.753846,0.793893


In [228]:
model_mlp_intro = mlp.load_model(dataset='intro', view='features')
model_mlp_mat = mlp.load_model(dataset='mat', view='features')
model_mlp_conc = mlp.load_model(dataset='conc', view='features')

In [247]:
model_mlp_intro = mlp.load_model(dataset='intro', view='features')
model_mlp_mat = mlp.load_model(dataset='mat', view='features')
model_mlp_conc = mlp.load_model(dataset='conc', view='features')

models = {'intro': model_mlp_intro, 'mat': model_mlp_mat, 'conc': model_mlp_conc}

summaries_mlp, intro_mlp, mat_mlp, conc_mlp = summarization.pipeline_summarization(
    'mlp', p, models, columns_name, summ_items, path_base, sections=['intro', 'mat', 'conc', 'concat'])

## Summaries

In [328]:
summaries['gb'][0].iloc[0]

articles                                              PMC2759846.json
references          Mutations in Leucine rich repeat kinase (LRRK2...
summaries_intro     Protein kinases play key regulatory roles for ...
summaries_mat       LRRK213262527 was cloned into pFastBac1 (Invit...
summaries_conc      Several mutations in the complex multi domain ...
summaries_concat    Protein kinases play key regulatory roles for ...
intro_r1                                                     0.180328
intro_r2                                                     0.033333
intro_rl                                                     0.147541
mat_r1                                                       0.092715
mat_r2                                                              0
mat_rl                                                       0.092715
conc_r1                                                      0.285714
conc_r2                                                      0.018182
conc_rl             

In [329]:
summaries['gb'][0].iloc[0]['references']

'Mutations in Leucine rich repeat kinase (LRRK2) are a common cause of inherited Parkinson’s disease (PD). The protein is large and complex, but pathogenic mutations cluster in a region containing GTPase and kinase domains. LRRK2 can autophosphorylate in vitro within a dimer pair, although the significance of this reaction is unclear. Here, we mapped the sites of autophosphorylation within LRRK2 and found several potential phosphorylation sites within the GTPase domain. Using mass spectrometry, we found that Thr1343 is phosphorylated and, using kinase dead versions of LRRK2, show that this is an autophosphorylation site. However, we also find evidence for additional sites in the GTPase domain and in other regions of the protein suggesting that there may be multiple autophosphorylation sites within LRRK2. These data suggest that the kinase and GTPase activities of LRRK2 may exhibit complex autoregulatory interdependence.'

In [330]:
summaries['gb'][0].iloc[0]['summaries_intro']

'Protein kinases play key regulatory roles for many processes in cells, influencing signal transduction cascades by the phosphorylation of target proteins. Proposed substrates include moesin , 4EBP , MKK3/6 , β tubulin and α synuclein but none have been shown to be true in vivo, and pathogenic mutations do not consistently have increased activity towards any specific substrate. Surprisingly, we found that LRRK2 autophosphorylates within the ROC/GTPase domain of the same protein, raising the possibility that complex autoregulation of LRRK2 may occur .'

In [331]:
summaries['gb'][0].iloc[0]['summaries_mat']

'LRRK213262527 was cloned into pFastBac1 (Invitrogen) with N terminal Flag tag and C terminal His tags and used to generate baculovirus with the Bac to Bac Expression system (Invitrogen). For mammalian expression, full length human LRRK2, the N terminal region of LRRK2 (amino acids 11248), LRR (9821280), ROC (13351548), COR (15501880), kinase (18802138) and WD40 (21502500) were cloned into the pCHMWS expression plasmid , incorporating an N terminal 3×flag tag. Recombinant domain proteins were incubated with recombinant GST tagged LRRK29702527 or recombinant full length 3×flag tagged LRRK2 in 40 µl kinase buffer containing 6 µCi of 33P ATP (3000 Ci/mmol; Perkin Elmer) for 1 hour at 30°C.'

In [332]:
summaries['gb'][0].iloc[0]['summaries_conc']

'Several mutations in the complex multi domain kinase LRRK2 are linked to PD but the mechanism(s) by which they affect protein function are poorly understood. Another minor site is at T2031, which we have previously shown can impact kinase activity as expected for an activation loop residue  . It is of interest that these sites are within the other enzymatic domain of LRRK2, the ROC/GTPase region as this suggests a complex autoregulation of LRRK2.'

### 300

In [81]:
summaries_gb.iloc[300]

articles                                             PMC2082664.json
references         We describe two spatially distinct foci of hum...
summaries_intro    Symptoms include general malaise, anemia, head...
summaries_mat      The Tororo, Iganga, Jinja and Busia Districts ...
summaries_conc     While in the present study, TGF β was not rela...
combinacao         Symptoms include general malaise, anemia, head...
intro_r1                                                    0.174603
intro_r2                                                         0.0
intro_rl                                                    0.095238
mat_r1                                                      0.071429
mat_r2                                                             0
mat_rl                                                      0.071429
conc_r1                                                     0.153846
conc_r2                                                     0.031088
conc_rl                           

In [82]:
summaries_gb.iloc[300]['references']

'We describe two spatially distinct foci of human African trypansomiasis in Eastern Uganda. The Tororo and Soroti foci of Trypansoma brucei rhodesiense infection were genetically distinct as characterised by microsatellite and minisatellite polymorphic markers, and were characterised by differences in disease progression and host immune response. In particular, infections with the Tororo genotype exhibited an increased frequency of progression to and severity of the meningoencephalitic stage and higher plasma IFN γ concentration compared to those with the Soroti genotype. We propose that the magnitude of the systemic IFN γ response determines the time at which infected individuals develop CNS infection, and this is consistent with the recently described role of IFN γ in facilitating blood brain barrier transmigration of trypanosomes in experimental model infection. The identification of trypanosome isolates with differing disease progression phenotypes provides the first field based ge

In [83]:
summaries_gb.iloc[300]['summaries_intro']

'Symptoms include general malaise, anemia, headache, pyrexia, weight loss and weakness. Typically T.b.rhodesiense infections are acute, while T.b.gambiense presents as a chronic disease . Both clinical and experimental animal studies have observed systemically high levels of IFN γ during trypanosome infection [ ], and following trypanosome invasion of the CNS there is direct relationship between the severity of neuropathology and expression of IFN γ in the brain  .'

In [84]:
summaries_gb.iloc[300]['summaries_mat']

'The Tororo, Iganga, Jinja and Busia Districts define a common ecotope for the transmission by Glossina fuscipes fuscipes of T.b.rhodesiense which will be referred to henceforth as the Tororo focus, while the Soroti District contains a separate G.f.fuscipes ecotope where HAT emerged as a new epidemic in 1998/9 . The Tororo, Iganga, Jinja and Busia Districts define a common ecotope for the transmission by Glossina fuscipes fuscipes of T.b.rhodesiense which will be referred to henceforth as the Tororo focus, while the Soroti District contains a separate G.f.fuscipes ecotope where HAT emerged as a new epidemic in 1998/9 . The Tororo, Iganga, Jinja and Busia Districts define a common ecotope for the transmission by Glossina fuscipes fuscipes of T.b.rhodesiense which will be referred to henceforth as the Tororo focus, while the Soroti District contains a separate G.f.fuscipes ecotope where HAT emerged as a new epidemic in 1998/9 .'

In [85]:
summaries_gb.iloc[300]['summaries_conc']

'While in the present study, TGF β was not related to disease severity, when compared to the cytokine responses reported in , both Tororo and Soroti patient plasma TGF β levels were lower than in Malawi HAT patients, while at the same time Tororo patient plasma IFN γ levels were significantly higher than those of Malawi patients, which were not significantly different to the Soroti patient plasma IFN γ concentration. While in the present study, TGF β was not related to disease severity, when compared to the cytokine responses reported in , both Tororo and Soroti patient plasma TGF β levels were lower than in Malawi HAT patients, while at the same time Tororo patient plasma IFN γ levels were significantly higher than those of Malawi patients, which were not significantly different to the Soroti patient plasma IFN γ concentration. While in the present study, TGF β was not related to disease severity, when compared to the cytokine responses reported in , both Tororo and Soroti patient pla

# MLP

## Summarization

In [136]:
def aux(models, datasets, predictions, y_true):

    results = {}
    
    for i in dataset.keys():
        
        knn, ab, gbc, rfc = models[i]
        X, _ = dataset[i]
        y = y_true[i]['bin']
        
        results['knn_{}'.format(i)] = create_df('knn_{}'.format(i), X, y, predictions['knn_{}'.format(i)])
        results['ab_{}'.format(i)] = create_df('ab_{}'.format(i), X, y, predictions['ab_{}'.format(i)])
        results['gb_{}'.format(i)] = create_df('gb_{}'.format(i), X, y, predictions['gb_{}'.format(i)])
        results['rf_{}'.format(i)] = create_df('rf_{}'.format(i), X, y, predictions['rf_{}'.format(i)])
        
    return results

In [138]:
def create_df(name_model, x_summ, y_true, y_pred):
    
    df = pd.DataFrame({'sentences': x_summ['sentences'],
                       'rouge_1': list(y_true),
                        name_model : y_pred.reshape(1, -1)[0],
                       'articles': x_summ['articles']})
        
    return df

In [142]:
def main(name_model, intro_result, mat_result, conc_result, summ_items, path_base):

    references = [summarization.get_ref_summary(i, path_base) for i in summ_items]
    
    summaries = summarization.create_summaries_df(name_model, intro_result, mat_result, conc_result, summ_items, references)
        
    summaries = summarization.evaluate_summaries(summaries, sections=['intro', 'mat', 'conc', 'comb'])
    
    return summaries

In [139]:
results = aux(models, dataset, predictions, y_true)

In [145]:
path_base = "../../sumdata/dataset_articles"
summaries = {}

name_models = ['knn', 'ab', 'rf', 'gb']

for i in name_models:

    summ = main(name_model=i, intro_result=results['{}_intro'.format(i)], mat_result=results['{}_mat'.format(i)], conc_result=results['{}_conc'.format(i)], summ_items=summ_items, path_base=path_base)
    summaries[i] = summ

# One Class Classification

In [161]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from src import utils_classification as utils
from sklearn.metrics import matthews_corrcoef

In [159]:
X_test = dataset['concat'][1]
y_test = dataset['concat'][3]

X_train = dataset['concat'][0]
y_train = dataset['concat'][2]

In [165]:
trainX = X_train[y_train==1]

nu = (sum(y_test))/trainX.shape[0]

svm = OneClassSVM(gamma='scale', nu=0.08)
rf = IsolationForest(contamination=0.08)

svm.fit(trainX)
rf.fit (trainX)

pred_svm = svm.predict(X_test)
pred_rf = rf.predict(X_test)

y_test[y_test == 1] = 1
y_test[y_test == 0] = -1

score = classification_report(y_test, pred_svm, labels=[-1, 1])
print( score)

score = classification_report(y_test, pred_rf,  labels=[-1, 1])
print(score)

              precision    recall  f1-score   support

          -1       1.00      0.11      0.19    127617
           1       0.00      0.00      0.00         0

    accuracy                           0.11    127617
   macro avg       0.50      0.05      0.10    127617
weighted avg       1.00      0.11      0.19    127617

              precision    recall  f1-score   support

          -1       1.00      0.07      0.13    127617
           1       0.00      0.00      0.00         0

    accuracy                           0.07    127617
   macro avg       0.50      0.03      0.06    127617
weighted avg       1.00      0.07      0.13    127617



In [ ]:
matthews_corrcoef(y_test, pred_svm)

In [ ]:
matthews_corrcoef(y_test, pred_rf)

In [40]:
pred_svm[pred_svm == -1] = 1
pred_svm[pred_svm == 1] = 0

In [41]:
pred_rf[pred_rf == -1] = 1
pred_rf[pred_rf == 1] = 0

In [42]:
sum(pred_svm)

0

In [30]:
sum(pred_rf)

0

In [259]:
y_test[y_test == -1] = 1
y_test[y_test == 1] = 0

In [54]:
X_test = X_summ_concat
y_test = y_summ_concat['bin']

In [56]:
y_concat_train

array([0, 0, 0, ..., 0, 0, 0])

In [57]:
import numpy as np
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

X_test = X_summ_concat.copy()
y_test = y_summ_concat.copy()['bin']

rbm = BernoulliRBM(n_components=100, learning_rate=0.01, random_state=42, verbose=True)
gb = GradientBoostingClassifier(n_estimators=100, max_depth=10, random_state=0)

rbm_features_classifier = Pipeline(
    steps=[('rbm', rbm), ('gb', gb)])

rbm_features_classifier.fit(X_concat_train, y_concat_train)

y_pred = rbm_features_classifier.predict(X_test)
print("Logistic regression using RBM features:\n%s\n" % (
    metrics.classification_report(y_test, y_pred)))

[BernoulliRBM] Iteration 1, pseudo-likelihood = -3932.93, time = 5.37s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -7809.93, time = 8.98s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -11667.83, time = 9.20s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -15515.23, time = 8.90s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -19355.68, time = 9.22s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -23190.91, time = 10.12s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -27021.96, time = 9.59s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -30849.52, time = 9.99s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -34674.14, time = 9.56s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -38496.26, time = 9.68s


KeyboardInterrupt: 

In [ ]:
rbm_features_classifier.

In [53]:
print("Logistic regression using RBM features:\n%s\n" % (
    metrics.classification_report(y_test, y_pred)))

Logistic regression using RBM features:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98    123652
           1       0.00      0.00      0.00      3965

    accuracy                           0.97    127617
   macro avg       0.48      0.50      0.49    127617
weighted avg       0.94      0.97      0.95    127617


